In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import os
import re
from typing import List, Dict
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
"""Initialize WebDriver with optimized settings"""
options = Options()
#options.add_argument("--headless")  # Remove this line if you want to see the browser
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-extensions")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36")
options.add_argument('--log-level=3')
options.page_load_strategy = 'eager'

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)
driver.set_page_load_timeout(30)
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

In [3]:
# Vai alla pagina dei giocatori
url = "https://www.nba.com/players"
driver.get(url)

In [4]:
decline_btn = WebDriverWait(driver, 3).until(
        EC.element_to_be_clickable((By.ID, "onetrust-reject-all-handler"))
    )
decline_btn.click()
print("✅ Cookie declinati")

time.sleep(5)

✅ Cookie declinati


In [5]:
# Aspetta che il toggle sia presente e cliccabile
try:
    toggle_label = WebDriverWait(driver, 3).until(
        EC.element_to_be_clickable((By.XPATH, "//label[contains(@class, 'Toggle_toggle') and contains(., 'Show Historic')]"))
    )
    toggle_label.click()
    print("✅ Toggle 'Show Historic' attivato")
except Exception as e:
    print("⚠️ Errore cliccando il toggle:", e)

time.sleep(5)

✅ Toggle 'Show Historic' attivato


In [6]:
def convert_height_to_meters(height_str):
    try:
        if '-' not in height_str:
            return None
        feet, inches = map(int, height_str.split('-'))
        total_meters = feet * 0.3048 + inches * 0.0254
        return round(total_meters, 2)
    except:
        return None

def convert_weight_to_kg(weight_str):
    try:
        lbs = int(weight_str.replace(" lbs", "").strip())
        return round(lbs * 0.453592, 1)
    except:
        return None

# Estrai tutte le righe della tabella
rows = driver.find_elements(By.CSS_SELECTOR, "table tbody tr")

players = []

for page in range(1, 103):
    print(f"📄 Pagina {page}")
    
    # Aspetta che la tabella sia presente
    WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "table tbody tr")))
    rows = driver.find_elements(By.CSS_SELECTOR, "table tbody tr")

    for row in rows:
        cols = row.find_elements(By.TAG_NAME, "td")
        if len(cols) >= 6:
            try:
                first_name = cols[0].find_element(By.CLASS_NAME, "RosterRow_playerFirstName__NYm50").text.strip()
                last_name = cols[0].find_elements(By.TAG_NAME, "p")[1].text.strip()
                full_name = f"{first_name} {last_name}"

                height = convert_height_to_meters(cols[4].text.strip())
                weight = convert_weight_to_kg(cols[5].text.strip())

                players.append({
                    "name": full_name,
                    "height": height,
                    "weight": weight
                })
            except Exception as e:
                print(f"⚠️ Errore nella riga: {e}")
                continue

    # Clicca su “Next Page” se presente
    try:
        next_btn = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@title='Next Page Button' and not(@disabled)]"))
        )
        next_btn.click()
        time.sleep(2)
    except:
        print("⛔ Fine pagine o bottone non cliccabile.")
        break

📄 Pagina 1
📄 Pagina 2
📄 Pagina 3
📄 Pagina 4
📄 Pagina 5
📄 Pagina 6
📄 Pagina 7
📄 Pagina 8
📄 Pagina 9
📄 Pagina 10
📄 Pagina 11
📄 Pagina 12
📄 Pagina 13
📄 Pagina 14
📄 Pagina 15
📄 Pagina 16
📄 Pagina 17
📄 Pagina 18
📄 Pagina 19
📄 Pagina 20
📄 Pagina 21
📄 Pagina 22
📄 Pagina 23
📄 Pagina 24
📄 Pagina 25
📄 Pagina 26
📄 Pagina 27
📄 Pagina 28
📄 Pagina 29
📄 Pagina 30
📄 Pagina 31
📄 Pagina 32
📄 Pagina 33
📄 Pagina 34
📄 Pagina 35
📄 Pagina 36
📄 Pagina 37
📄 Pagina 38
📄 Pagina 39
📄 Pagina 40
📄 Pagina 41
📄 Pagina 42
📄 Pagina 43
📄 Pagina 44
📄 Pagina 45
📄 Pagina 46
📄 Pagina 47
📄 Pagina 48
📄 Pagina 49
📄 Pagina 50
📄 Pagina 51
📄 Pagina 52
📄 Pagina 53
📄 Pagina 54
📄 Pagina 55
📄 Pagina 56
📄 Pagina 57
📄 Pagina 58
📄 Pagina 59
📄 Pagina 60
📄 Pagina 61
📄 Pagina 62
📄 Pagina 63
📄 Pagina 64
📄 Pagina 65
📄 Pagina 66
📄 Pagina 67
📄 Pagina 68
📄 Pagina 69
📄 Pagina 70
📄 Pagina 71
📄 Pagina 72
📄 Pagina 73
📄 Pagina 74
📄 Pagina 75
📄 Pagina 76
📄 Pagina 77
📄 Pagina 78
📄 Pagina 79
📄 Pagina 80
📄 Pagina 81
📄 Pagina 82
📄 Pagina 83
📄 Pagina 84
📄

In [9]:
players

[{'name': 'Alaa Abdelnaby', 'height': 2.08, 'weight': 108.9},
 {'name': 'Zaid Abdul-Aziz', 'height': 2.06, 'weight': 106.6},
 {'name': 'Kareem Abdul-Jabbar', 'height': 2.18, 'weight': 102.1},
 {'name': 'Mahmoud Abdul-Rauf', 'height': 1.85, 'weight': 73.5},
 {'name': 'Tariq Abdul-Wahad', 'height': 1.98, 'weight': 106.6},
 {'name': 'Shareef Abdur-Rahim', 'height': 2.06, 'weight': 111.1},
 {'name': 'Tom Abernethy', 'height': 2.01, 'weight': 99.8},
 {'name': 'Forest Able', 'height': 1.91, 'weight': 81.6},
 {'name': 'John Abramovic', 'height': 1.91, 'weight': 88.5},
 {'name': 'Alex Abrines', 'height': 1.98, 'weight': 86.2},
 {'name': 'Precious Achiuwa', 'height': 2.03, 'weight': 110.2},
 {'name': 'Alex Acker', 'height': 1.96, 'weight': 83.9},
 {'name': 'Donald Ackerman', 'height': 1.83, 'weight': 83.0},
 {'name': 'Mark Acres', 'height': 2.11, 'weight': 99.8},
 {'name': 'Charles Acton', 'height': 1.98, 'weight': 95.3},
 {'name': 'Quincy Acy', 'height': 2.01, 'weight': 108.9},
 {'name': 'Alva

In [10]:
len(players)

5084

In [11]:
pd.DataFrame(players).to_csv("nba_players_all.csv", index=False)

In [14]:
# File database giocatori
df = pd.read_csv('Scraping_data/2_players_database.csv')
df

,Nome,Cognome,Nome_Completo
0,OG,Anunoby,OG Anunoby
1,Josh,Hart,Josh Hart
2,Karl-Anthony,Towns,Karl-Anthony Towns
3,Mikal,Bridges,Mikal Bridges
4,Jalen,Brunson,Jalen Brunson
...,...,...,...
573,Phillip,Wheeler,Phillip Wheeler
574,Drew,Timme,Drew Timme
575,Jaylen,Sims,Jaylen Sims
576,Terence,Davis,Terence Davis


In [16]:
from collections import Counter

# Estrai solo i nomi
names = [p["name"] for p in players]

# Conta i nomi
name_counts = Counter(names)

# Filtra solo quelli con più di 1 occorrenza
duplicates = {name: count for name, count in name_counts.items() if count > 1}

# Mostra risultati
if duplicates:
    print("🔁 Nomi duplicati trovati:")
    for name, count in duplicates.items():
        print(f"- {name}: {count} volte")
else:
    print("✅ Nessun nome duplicato trovato.")

🔁 Nomi duplicati trovati:
- Dee Brown: 2 volte
- Charlie Davis: 2 volte
- Johnny Davis: 2 volte
- Mark Davis: 2 volte
- Bob Duffy: 2 volte
- Mike Dunleavy: 2 volte
- Patrick Ewing: 2 volte
- Matt Guokas: 2 volte
- Cedric Henderson: 2 volte
- Gerald Henderson: 2 volte
- Mike James: 2 volte
- Chris Johnson: 2 volte
- David Johnson: 2 volte
- Eddie Johnson: 2 volte
- George Johnson: 3 volte
- Ken Johnson: 2 volte
- Larry Johnson: 2 volte
- Bobby Jones: 2 volte
- Charles Jones: 2 volte
- Mark Jones: 2 volte
- Willie Jones: 2 volte
- George King: 2 volte
- Tony Mitchell: 2 volte
- Jim Paxson: 2 volte
- Glen Rice: 2 volte
- Walker Russell: 2 volte
- Bill Smith: 2 volte
- Charles Smith: 3 volte
- Chris Smith: 2 volte
- Greg Smith: 2 volte
- Michael Smith: 2 volte
- Steven Smith: 2 volte
- Jack Turner: 2 volte
- Brandon Williams: 2 volte
- Marcus Williams: 2 volte
- Nate Williams: 2 volte
- Reggie Williams: 2 volte
- Chris Wright: 2 volte


In [19]:
# Step 1: Converti players in DataFrame
players_df = pd.DataFrame(players)  # deve contenere "name", "height", "weight"

# Step 2: Merge con il tuo df originale
df_merged = df.merge(players_df, how='left', left_on='Nome_Completo', right_on='name')

# Step 3: Pulisci (puoi rimuovere la colonna 'name' se è ridondante)
df_merged.drop(columns=['name'], inplace=True)

# Visualizza o salva
print(df_merged.head())
df_merged.to_csv("Scraping_data/players_with_physical.csv", index=False)


           Nome  Cognome       Nome_Completo  height  weight
0            OG  Anunoby          OG Anunoby    2.01   108.9
1          Josh     Hart           Josh Hart    1.93    97.5
2  Karl-Anthony    Towns  Karl-Anthony Towns    2.13   112.5
3         Mikal  Bridges       Mikal Bridges    1.98    94.8
4         Jalen  Brunson       Jalen Brunson    1.88    86.2


In [18]:
missing = df_merged[df_merged['height'].isna()]
print("❌ Giocatori senza dati trovati:", len(missing))
print(missing[['Nome_Completo']])

❌ Giocatori senza dati trovati: 5
      Nome_Completo
438      Taze Moore
441    Armel Traore
514    Boo Buie III
548       Dink Pate
551  Erik Stevenson
